Both kernel ridge regression (KRR) and SVR learn a non-linear function by employing the kernel trick, i.e., they learn a linear function in the space induced by the respective kernel which corresponds to a non-linear function in the original space. They differ in the loss functions (ridge versus epsilon-insensitive loss). In contrast to SVR, fitting a KRR can be done in closed-form and is typically faster for medium-sized datasets. On the other hand, the learned model is non-sparse and thus slower than SVR at prediction-time.

This example illustrates both methods on an artificial dataset, which consists of a sinusoidal target function and strong noise added to every fifth datapoint. The first figure compares the learned model of KRR and SVR when both complexity/regularization and bandwidth of the RBF kernel are optimized using grid-search. The learned functions are very similar; however, fitting KRR is approx. seven times faster than fitting SVR (both with grid-search). However, prediction of 100000 target values is more than tree times faster with SVR since it has learned a sparse model using only approx. 1/3 of the 100 training datapoints as support vectors.

The next figure compares the time for fitting and prediction of KRR and SVR for different sizes of the training set. Fitting KRR is faster than SVR for medium- sized training sets (less than 1000 samples); however, for larger training sets SVR scales better. With regard to prediction time, SVR is faster than KRR for all sizes of the training set because of the learned sparse solution. Note that the degree of sparsity and thus the prediction time depends on the parameters epsilon and C of the SVR.

#### New to Plotly?
Plotly's Python library is free and open source! [Get started](https://plot.ly/python/getting-started/) by downloading the client and [reading the primer](https://plot.ly/python/getting-started/).
<br>You can set up Plotly to work in [online](https://plot.ly/python/getting-started/#initialization-for-online-plotting) or [offline](https://plot.ly/python/getting-started/#initialization-for-offline-plotting) mode, or in [jupyter notebooks](https://plot.ly/python/getting-started/#start-plotting-online).
<br>We also have a quick-reference [cheatsheet](https://images.plot.ly/plotly-documentation/images/python_cheat_sheet.pdf) (new!) to help you get started!

### Version

In [1]:
import sklearn
sklearn.__version__

'0.18'

### Imports

This tutorial imports [SVR](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html#sklearn.svm.SVR), [GridSearchCV](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) and [KernelRidge](http://scikit-learn.org/stable/modules/generated/sklearn.kernel_ridge.KernelRidge.html#sklearn.kernel_ridge.KernelRidge)

In [2]:
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

from __future__ import division
import time
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge

### Calculatons

Generate sample data

In [3]:
rng = np.random.RandomState(0)
X = 5 * rng.rand(10000, 1)
y = np.sin(X).ravel()

# Add noise to targets
y[::5] += 3 * (0.5 - rng.rand(X.shape[0]/5))

X_plot = np.linspace(0, 5, 100000)[:, None]

/home/diksha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future



### Fit regression model

In [4]:
train_size = 100
svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
                   param_grid={"C": [1e0, 1e1, 1e2, 1e3],
                               "gamma": np.logspace(-2, 2, 5)})

kr = GridSearchCV(KernelRidge(kernel='rbf', gamma=0.1), cv=5,
                  param_grid={"alpha": [1e0, 0.1, 1e-2, 1e-3],
                              "gamma": np.logspace(-2, 2, 5)})

t0 = time.time()
svr.fit(X[:train_size], y[:train_size])
svr_fit = time.time() - t0
print("SVR complexity and bandwidth selected and model fitted in %.3f s"
      % svr_fit)

t0 = time.time()
kr.fit(X[:train_size], y[:train_size])
kr_fit = time.time() - t0
print("KRR complexity and bandwidth selected and model fitted in %.3f s"
      % kr_fit)

sv_ratio = svr.best_estimator_.support_.shape[0] / train_size
print("Support vector ratio: %.3f" % sv_ratio)

t0 = time.time()
y_svr = svr.predict(X_plot)
svr_predict = time.time() - t0
print("SVR prediction for %d inputs in %.3f s"
      % (X_plot.shape[0], svr_predict))

t0 = time.time()
y_kr = kr.predict(X_plot)
kr_predict = time.time() - t0
print("KRR prediction for %d inputs in %.3f s"
      % (X_plot.shape[0], kr_predict))

SVR complexity and bandwidth selected and model fitted in 0.898 s
KRR complexity and bandwidth selected and model fitted in 0.431 s
Support vector ratio: 0.320
SVR prediction for 100000 inputs in 0.138 s
KRR prediction for 100000 inputs in 0.312 s


### Plot Results

In [5]:
def to_plotly(n):
    l = []
    for i in range(len(n)):
        k=float(n[i])
        l.append( k)
    return l
fig = tools.make_subplots(rows=2, cols=2,specs=[[{}, {}],
                                 [{}, None],],
                          subplot_titles=('SVR versus Kernel Ridge', 
                                                         'Execution Time',
                                                          'Learning Curves',))

sv_ind = svr.best_estimator_.support_

trace1 = go.Scatter(x=to_plotly(X[sv_ind]),
                    y=to_plotly(y[sv_ind]), 
                    name='SVR support vectors',
                    mode='markers',
                    marker=dict(
                            color='red', size=10,
                            line=dict(
                                color='black', width=1))
                   )
trace2 = go.Scatter(x = to_plotly(X[:100]),
                    y= to_plotly(y[:100]),
                    name='data', mode='markers', 
                    marker=dict(
                                color='green',size=5,
                                line=dict(
                                        color='black'))
                   )
trace3 = go.Scatter(x=to_plotly(X_plot), y=to_plotly(y_svr),
                    marker=dict(
                            line=dict(color='red')),
                            mode='lines',
                            name='SVR (fit: %.3fs, predict: %.3fs)' % (svr_fit, svr_predict)
                   )
trace4 = go.Scatter(x=to_plotly(X_plot), y=to_plotly(y_kr),
                    marker=dict(
                                line=dict(color='green')),
                    mode='lines',
                    name='KRR (fit: %.3fs, predict: %.3fs)' % (kr_fit, kr_predict)
                   )

fig['layout']['xaxis1'].update(title='data')
fig['layout']['yaxis1'].update(title='target')

for i in [trace1,trace2,trace3,trace4]:
    fig.append_trace(i , 1, 1)

# Visualize training and prediction time

# Generate sample data
X = 5 * rng.rand(10000, 1)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(X.shape[0]/5))
sizes = np.logspace(1, 4, 7)
plot2=[]

for name, estimator in {"KRR": KernelRidge(kernel='rbf', alpha=0.1,
                                           gamma=10),
                        "SVR": SVR(kernel='rbf', C=1e1, gamma=10)}.items():
    train_time = []
    test_time = []
    for train_test_size in sizes:
        t0 = time.time()
        estimator.fit(X[:train_test_size], y[:train_test_size])
        train_time.append(time.time() - t0)

        t0 = time.time()
        estimator.predict(X_plot[:1000])
        test_time.append(time.time() - t0)

    trace5 = go.Scatter(x=sizes, y=train_time,
                        mode='lines+markers',
                        marker=dict(
                                color="red" if name == "SVR" else "green"),
                                name="%s (train)" % name)
    trace6 = go.Scatter(x=sizes, y=test_time, 
                        mode='lines+markers',
                        marker=dict(
                                color="red" if name == "SVR" else "green"),
                        line=dict(dash = 'dash'),
                        name="%s (test)" % name)
    plot2.append(trace5)
    plot2.append(trace6)


for i in range(len(plot2)):
     fig.append_trace(plot2[i] , 1, 2)
    
fig['layout']['xaxis2'].update(title='Train size',type='log')
fig['layout']['yaxis2'].update(title='Time (seconds)',type='log')

# Visualize learning curves

svr = SVR(kernel='rbf', C=1e1, gamma=0.1)
kr = KernelRidge(kernel='rbf', alpha=0.1, gamma=0.1)
train_sizes, train_scores_svr, test_scores_svr = \
    learning_curve(svr, X[:100], y[:100], train_sizes=np.linspace(0.1, 1, 10),
                   scoring="neg_mean_squared_error", cv=10)
train_sizes_abs, train_scores_kr, test_scores_kr = \
    learning_curve(kr, X[:100], y[:100], train_sizes=np.linspace(0.1, 1, 10),
                   scoring="neg_mean_squared_error", cv=10)

trace7 = go.Scatter(x=train_sizes, y=-test_scores_svr.mean(1), 
                    mode='lines+markers',
                    marker=dict(color="red",),
                    name="SVR")
trace8 = go.Scatter(x=train_sizes, y=-test_scores_kr.mean(1), 
                    mode='lines+markers',
                    marker=dict(color="green",),
                    name="KRR")

fig.append_trace(trace7 , 2, 1)
fig.append_trace(trace8 , 2, 1)
                    
fig['layout']['xaxis3'].update(title='Train size')
fig['layout']['yaxis3'].update(title='Mean Squared Error')
fig['layout'].update(height = 900)


This is the format of your plot grid:
[ (1,1) x1,y1 ]  [ (1,2) x2,y2 ]
[ (2,1) x3,y3 ]      (empty)    



/home/diksha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:56: VisibleDeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future

/home/diksha/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:67: VisibleDeprecationWarning:

using a non-integer number instead of an integer will result in an error in the future



In [6]:
py.iplot(fig, filename='comparision')

/home/diksha/.local/lib/python2.7/site-packages/plotly/plotly/plotly.py:236: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



The draw time for this plot will be slow for all clients.


/home/diksha/.local/lib/python2.7/site-packages/plotly/plotly/plotly.py:1443: UserWarning:

Estimated Draw Time Too Long



### License

Authors: 
            
            Jan Hendrik Metzen <jhm@informatik.uni-bremen.de>
License: 
            
            BSD 3 clause

In [2]:
from IPython.display import display, HTML

display(HTML('<link href="//fonts.googleapis.com/css?family=Open+Sans:600,400,300,200|Inconsolata|Ubuntu+Mono:400,700" rel="stylesheet" type="text/css" />'))
display(HTML('<link rel="stylesheet" type="text/css" href="http://help.plot.ly/documentation/all_static/css/ipython-notebook-custom.css">'))

! pip install git+https://github.com/plotly/publisher.git --upgrade
import publisher
publisher.publish(
    'comparision.ipynb', 'scikit-learn/plot-kernel-ridge-regression/', 'Plot kernel ridge regression| plotly',
    ' ',
    title = 'Plot kernel ridge regression | plotly',
    name = 'Comparison of Kernel Ridge Regression and SVR',
    has_thumbnail='true', thumbnail='thumbnail/kernel-ridge.jpg', 
    language='scikit-learn', page_type='example_index',
    display_as='general_examples', order=8,ipynb='~Diksha_Gabha/2659')  

  Cloning https://github.com/plotly/publisher.git to /tmp/pip-WGlmWt-build
  Running setup.py install for publisher ... - error
    Complete output from command /usr/bin/python -u -c "import setuptools, tokenize;__file__='/tmp/pip-WGlmWt-build/setup.py';exec(compile(getattr(tokenize, 'open', open)(__file__).read().replace('\r\n', '\n'), __file__, 'exec'))" install --record /tmp/pip-DfImZj-record/install-record.txt --single-version-externally-managed --compile:
    running install
    running build
    running build_py
    creating build
    creating build/lib.linux-x86_64-2.7
    creating build/lib.linux-x86_64-2.7/publisher
    copying publisher/publisher.py -> build/lib.linux-x86_64-2.7/publisher
    copying publisher/__init__.py -> build/lib.linux-x86_64-2.7/publisher
    running install_lib
    creating /usr/local/lib/python2.7/dist-packages/publisher
    error: could not create '/usr/local/lib/python2.7/dist-packages/publisher': Permission denied
    
    -----------------------